# 📓 Mini RAG Notebook — IBM Report Example
This notebook demonstrates how to build a simple Retrieval-Augmented Generation (RAG) pipeline using an IBM report PDF.

## 1. Install Dependencies

In [ ]:
!pip install openai faiss-cpu numpy requests pypdf tiktoken ipywidgets

## 2. Configurations

In [ ]:
from openai import OpenAI
import getpass

# === Configuration ===
PDF_URL = "https://www.ibm.com/downloads/documents/us-en/1227c12d3a38b173"  # IBM 2024 Annual Report
CHAT_MODEL = "cerebras/llama-3.3-70b"        # choose: ""
EMBED_MODEL = "openai/text-embedding-ada-002"  # or ""

BASE_URL = "https://ca-tor.ml.cloud.ibm.com/ml/gateway/v1"  # or your custom endpoint
API_KEY = getpass.getpass("Enter your API key: ")

client = OpenAI(api_key=API_KEY, base_url=BASE_URL)

## 3. Download + Extract PDF Text

In [ ]:
import requests
from pathlib import Path
from pypdf import PdfReader

pdf_path = Path("ibm_report.pdf")
if not pdf_path.exists():
    print("Downloading PDF...")
    pdf_path.write_bytes(requests.get(PDF_URL).content)

reader = PdfReader(str(pdf_path))
text = ""
for page in reader.pages:
    if page.extract_text():
        text += page.extract_text() + "\n"

print("Preview of text:\n", text[:800])

## 4. Chunking the Text

In [ ]:
import tiktoken

def chunk_text(text, chunk_size=500, overlap=50):
    encoder = tiktoken.get_encoding("cl100k_base")
    tokens = encoder.encode(text)
    chunks = []
    for i in range(0, len(tokens), chunk_size - overlap):
        chunk = tokens[i:i+chunk_size]
        chunks.append(encoder.decode(chunk))
    return chunks

chunks = chunk_text(text, chunk_size=500, overlap=50)
print(f"Total chunks: {len(chunks)}")
print("Sample chunk:\n", chunks[0][:300])

## 5. Build FAISS Index with Embeddings

In [ ]:
import numpy as np
import faiss

# Batch function
def embed_chunks(chunks, batch_size=50):
    all_embeddings = []
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i+batch_size]
        resp = client.embeddings.create(model=EMBED_MODEL, input=batch)
        all_embeddings.extend([d.embedding for d in resp.data])
    return np.array(all_embeddings, dtype="float32")

# Run embedding in batches
embeddings = embed_chunks(chunks, batch_size=50)

# Build FAISS index
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

print("FAISS index ready with", index.ntotal, "vectors.")


## 6. Retrieval Helper

In [ ]:
def retrieve(query, k=3):
    q_emb = client.embeddings.create(model=EMBED_MODEL, input=query).data[0].embedding
    D, I = index.search(np.array([q_emb]).astype("float32"), k=k)
    return [chunks[i] for i in I[0]]

## 7. Ask Function (Chat with RAG)

In [ ]:
def ask(query, k=3):
    retrieved = retrieve(query, k=k)
    context = "\n\n".join(retrieved)
    prompt = f"Answer using the context below:\n{context}\n\nUser question: {query}"

    response = client.chat.completions.create(
        model=CHAT_MODEL,
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

# Example usage
print(ask("What are IBM’s AI initiatives mentioned in this report?"))

## 8. Interactive Q&A (Buttons + Chatbox)

You can now explore the IBM report in two ways:

- 🔘 **Click a button** to run a predefined “starter” question.  
- 💬 **Type your own question** into the chatbox below.  

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Predefined starter questions
questions = [
    "What are IBM’s AI initiatives mentioned in this report?",
    "What are IBM’s main strategic priorities for 2024?",
    "What risks to growth does IBM identify?",
    "What does the report say about sustainability?",
    "Which industries are adopting AI according to the report?"
]

output_box = widgets.Output()

# Button click handler
def on_button_click(b):
    with output_box:
        clear_output(wait=True)
        print(f"❓ {b.description}\n")
        print("💡 Answer:\n")
        print(ask(b.description))
        print("-" * 50)

# Create and display buttons
button_box = widgets.VBox([
    widgets.Button(description=q, layout=widgets.Layout(width="auto"))
    for q in questions
])

for btn in button_box.children:
    btn.on_click(on_button_click)

# Free-form input
input_box = widgets.Text(
    value='',
    placeholder='Type your question here...',
    description='Ask:',
    layout=widgets.Layout(width="80%")
)

def on_enter(change):
    if change['name'] == 'value' and change['new']:
        query = change['new']
        with output_box:
            print(f"❓ {query}\n")
            print("💡 Answer:\n")
            print(ask(query))
            print("-" * 50)
        input_box.value = ""  # clear input after submit

input_box.observe(on_enter)

# Display everything
display(widgets.Label("🔹 Click a starter question:"), button_box)
display(widgets.Label("🔹 Or type your own:"), input_box, output_box)
